# Unit 2: Fisheries Collapse Module Overview

This module will focus on examining a crucial global issue and important scientific debate about the state of global fisheries.  In this module we will seek to reproduce some of the most widely cited examples of species collapse ever, and examine the evidence behind an influential and widely cited paper on global fisheries, [Worm et al 2006](http://doi.org/10.1126/science.1132294).  However, rather than use the limited data available to Boris Worm and colleagues in 2006, we will be drawing from the best and most recent stock asssement data available today to see how those patterns have faired.  

In this module we will also begin to master one of the most important concepts in data science: manipulation of tabular data using relational database concepts. Instead of working with independent data.frames, we will be working with a large relational database which contains many different tables of different sizes and shapes, but that all all related to each other through a series of different ids.  



## The Database

We will use data from the RAM Legacy Stock Assessment Database.  In order to better introduce some important emerging technologies, we will be accessing these data directly from a relatively new platform that is now playing a key role in data sharing in machine learning communities, with the memorable name, HuggingFace.  We will be streaming data from <https://huggingface.co/datasets/cboettig/ram_fisheries/tree/main/v4.65>.  We will have more to say about this approach as we progress.



## Researcher Spotlight: Daniel Pauly

Science is done by real people.  There are many influential and colorful characters in the global fisheries debate.  I want to highlight Professor Pauly not just because he is so famous, but as an early believer in Open Science and Data Science, before we had either of those words.  His contributions in making fisheries data more open were ground breaking for their time.  I'm also indebted to Professor Pauly whom I had the privilege to meet when I was a junior scientist who had only recently released one of my first software packages, aimed at making data from FishBase more accessible. Academic researchers are typically defined by scientific publications, not software, so I was shocked that Pauly already knew of my software package, and that he encouraged me to continue developing software.  Even today that is not common advice, but I believed him, and it's probably a good reason I am where I am today.  Scientific textbooks and courses are often critiqued for failing to recognize the contributions of those from minority backgrounds, but as the texts are written on global change ecology, I think none will omit the works for Professor Pauly.




## Science Introduction

Background abbreviated documentary, features many of the leading authors on both sides https://vimeo.com/44104959

In [1]:
import ibis
from ibis import _
import ibis.selectors as s
import seaborn.objects as so


# Exercise 1: Investigating the North-Atlantic Cod

Now we are ready to dive into our data. First, We seek to replicate the following figure from the Millennium Ecosystem Assessment Project using the RAM data.

![](https://espm-157.github.io/website-r/img/cod.jpg)


In [4]:
con=ibis.duckdb.connect() # Connect to duckDB to download the data

area_url = "https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/area.csv?download=true"
area = con.read_csv(area_url)

assessment_url = "https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/assessment.csv?download=true"
assessment = con.read_csv(assessment_url)

stock_url = "https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/stock.csv?download=true"
stock = con.read_csv(stock_url)

time_series_url = "https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/timeseries.csv?download=true"
time_series = con.read_csv(time_series_url)

metrics_url = "https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/tsmetrics.csv?download=true"
metrics = con.read_csv(metrics_url)

In [5]:
time_series.head().execute() # view part of the downloaded data

,assessid,stockid,stocklong,tsid,tsyear,tsvalue
0,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,Blue grenadier Southeast Australia,CdivMEANC-ratio,1960,NA
1,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,Blue grenadier Southeast Australia,CdivMEANC-ratio,1961,NA
2,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,Blue grenadier Southeast Australia,CdivMEANC-ratio,1962,NA
3,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,Blue grenadier Southeast Australia,CdivMEANC-ratio,1963,NA
4,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,Blue grenadier Southeast Australia,CdivMEANC-ratio,1964,NA


In [ ]:
stock.head().execute() # view the stock data

,stockid,tsn,scientificname,commonname,areaid,stocklong,region,primary_country,primary_FAOarea,ISO3_code,GRSF_uuid,GRSF_areaid,inmyersdb,myersstockid,state
0,ACADRED2J3K,166774,Sebastes fasciatus,Acadian redfish,Canada-DFO-2J3K,Acadian redfish NAFO Division 2J3K,Canada East Coast,Russian Federation,21,RUS,6ef91e5a-4a51-3ff8-bfc9-065cba1a5f79,fao:21.2.J;fao:21.3.K,0,NA,Current
1,ACADRED3LNO-UT12,166774,Sebastes fasciatus,Acadian redfish,Canada-DFO-3LNO-UT12,Acadian redfish NAFO Division 3LNO and Units 1...,Canada East Coast,Canada,21,CAN,6d430af8-0240-3b76-a041-b1848a405267,fao:21.3.L;fao:21.3.N;fao:21.3.O;fao:21.3.P.n;...,0,NA,Deprecated
2,ACADREDGOMGB,166774,Sebastes fasciatus,Acadian redfish,USA-NMFS-5YZ,Acadian redfish Gulf of Maine / Georges Bank,US East Coast,USA,21,USA,1dcb5911-42f2-3361-a031-45658fd864db,fao:21.5.Y;fao:21.5.Zu,0,NA,Current
3,ACADREDUT3,166774,Sebastes fasciatus,Acadian redfish,Canada-DFO-UT3,Acadian redfish Unit 3,Canada East Coast,Canada,21,CAN,5510fc9b-3a3c-30fb-aa2c-a2a389614daa,fao:21.4.W;fao:21.4.X,0,NA,Current
4,ACMACKSARG,172413,Scomber colias,Argentine chub mackerel,Argentina-CFP-ARG-S,Argentine chub mackerel Southern Argentina,South America,Argentina,41,ARG,58ff70a6-6e36-31a0-9f2f-3816c4e2aaaa,grsf:ARG_EEZ_39S_VMA,0,NA,Current


In [8]:
area.head().execute() # view the area data

,country,areatype,areacode,areaname,alternateareaname,areaid
0,Argentina,CFP,ARG-N,Northern Argentina,NA,Argentina-CFP-ARG-N
1,Argentina,CFP,ARG-S,Southern Argentina,NA,Argentina-CFP-ARG-S
2,Australia,AFMA,CASCADE,Cascade Plateau,NA,Australia-AFMA-CASCADE
3,Australia,AFMA,EAUS,Eastern Australia,NA,Australia-AFMA-EAUS
4,Australia,AFMA,ESE,Eastern half of Southeast Australia,NA,Australia-AFMA-ESE


Let's reproduce the graph shown at the start of the notebook.

In [11]:
stock.filter(_.commonname == "Atlantic cod").execute() # view the atlantic cod fish stock

,stockid,tsn,scientificname,commonname,areaid,stocklong,region,primary_country,primary_FAOarea,ISO3_code,GRSF_uuid,GRSF_areaid,inmyersdb,myersstockid,state
0,COD1ABCDE,164712,Gadus morhua,Atlantic cod,multinational-ICES-1ABCDE,Atlantic cod NAFO 1ABCDE,Canada East Coast,Greenland,21,GRL,01203eac-f124-34d5-80c0-44ce978b328e,fao:21.1.A;fao:21.1.B;fao:21.1.C;fao:21.1.D;fa...,0,NA,Current
1,COD1F-XIV,164712,Gadus morhua,Atlantic cod,multinational-ICES-1F-XIV,Atlantic cod NAFO 1F and ICES 14,Europe non EU,Greenland,27,GRL,269f6c1e-e0cd-3fcb-b2cc-41d338ac824e,fao:21.1.F;fao:27.14,0,NA,Current
2,COD1IN,164712,Gadus morhua,Atlantic cod,multinational-ICES-1IN,Atlantic cod NAFO Subarea 1 (inshore),Canada East Coast,Greenland,21,GRL,437d0699-5695-336d-b139-bc69de390334,fao:21.1,0,NA,Current
3,COD2J3KL,164712,Gadus morhua,Atlantic cod,Canada-DFO-2J3KL,Atlantic cod Southern Labrador-Eastern Newfoun...,Canada East Coast,Canada,21,CAN,c9055c35-a306-3a63-83b5-525d9f484840,fao:21.2.J;fao:21.3.K;fao:21.3.L,1,COD2J3KL,Current
4,COD3M,164712,Gadus morhua,Atlantic cod,multinational-NAFO-3M,Atlantic cod Flemish Cap,Canada East Coast,Portugal,21,PRT,ad76fd62-8f75-33d0-82cd-380f9c9f2533,fao:21.3.M,1,COD3M,Current
5,COD3NO,164712,Gadus morhua,Atlantic cod,multinational-NAFO-3NO,Atlantic cod Southern Grand Banks,Canada East Coast,Spain,21,ESP,f8bd5149-f25b-3377-850a-0de697a9bfb9,fao:21.3.N;fao:21.3.O,1,COD3NO,Current
6,COD3Pn4RS,164712,Gadus morhua,Atlantic cod,Canada-DFO-3Pn4RS,Atlantic cod Northern Gulf of St. Lawrence,Canada East Coast,Canada,21,CAN,5a19ce3d-ecba-3ee1-9f81-c766b3d18d52,fao:21.3.P.n;fao:21.4.R;fao:21.4.S,1,COD3Pn4RS,Current
7,COD3Ps,164712,Gadus morhua,Atlantic cod,Canada-DFO-3Ps,Atlantic cod St. Pierre Bank,Canada East Coast,Canada,21,CAN,44bdf775-5ab2-36dc-9781-1c5d774d92bd,fao:21.3.P.s,1,COD3Ps,Current
8,COD4TVn,164712,Gadus morhua,Atlantic cod,Canada-DFO-4T,Atlantic cod Southern Gulf of St. Lawrence,Canada East Coast,Canada,21,CAN,f0b7a32d-ba51-3028-941d-e18008450d12,fao:21.4.T,1,COD4TVn,Current
9,COD4VsW,164712,Gadus morhua,Atlantic cod,Canada-DFO-4VsW,Atlantic cod Eastern Scotian Shelf,Canada East Coast,Canada,21,CAN,4e12fcb9-141a-3eef-af10-23f19f4e4f11,fao:21.4.V.s;fao:21.4.W,1,COD4VsW,Current


In [12]:
time_series.filter(_.stockid == "COD1ABCDE").head().execute()

,assessid,stockid,stocklong,tsid,tsyear,tsvalue
0,NWWG-COD1ABCDE-1952-2015-ICESIMP2018,COD1ABCDE,Atlantic cod NAFO 1ABCDE,CdivMEANC-ratio,1952,1.31171349447347
1,NWWG-COD1ABCDE-1952-2015-ICESIMP2018,COD1ABCDE,Atlantic cod NAFO 1ABCDE,CdivMEANC-ratio,1953,2.01802076072842
2,NWWG-COD1ABCDE-1952-2015-ICESIMP2018,COD1ABCDE,Atlantic cod NAFO 1ABCDE,CdivMEANC-ratio,1954,2.99339746174715
3,NWWG-COD1ABCDE-1952-2015-ICESIMP2018,COD1ABCDE,Atlantic cod NAFO 1ABCDE,CdivMEANC-ratio,1955,2.70190557408638
4,NWWG-COD1ABCDE-1952-2015-ICESIMP2018,COD1ABCDE,Atlantic cod NAFO 1ABCDE,CdivMEANC-ratio,1956,3.3185230287534


In [13]:
# Let's assess how many unique values of tsid are there

time_series.filter(_.stockid == "COD1ABCDE").select(_.tsid).distinct().head().execute()

,tsid
0,TL-MT
1,TCbest-MT
2,CdivMEANC-ratio
3,TC-MT


In [15]:
metrics.filter(_.tsunique == "CdivMSY-ratio").execute()

,tscategory,tsshort,tslong,tsunitsshort,tsunitslong,tsunique
0,CATCH or LANDINGS,CdivMSY,Catch divided by MSY,ratio,ratio,CdivMSY-ratio


In [16]:
metrics.select(_.tscategory).distinct().execute() # distinct measurements in the data

,tscategory
0,CATCH or LANDINGS
1,FISHING MORTALITY
2,OTHER TIME SERIES DATA
3,SPAWNING STOCK BIOMASS or CPUE
4,PRODUCTION
5,TOTAL BIOMASS
6,RECRUITS (NOTE: RECRUITS ARE OFFSET IN TIME SE...
7,TIME UNITS


In [17]:
metrics.filter(_.tscategory == "CATCH or LANDINGS").select(_.tslong, _.tsunique).distinct().execute()

,tslong,tsunique
0,Scientific advice for catch limit (pairs with ...,Cadvised-CMB-MT
1,Catch divided by model version only MSY,CdivMSY-mvo-ratio
2,Customary catch,CUSTC-MT
3,Recreational catch,RecC-MT
4,Total allowable catch,TAC-2-MT
...,...,...
63,Total catch. Use only when there is more than ...,TC-3-E06
64,Total catch (i.e. landings + discards. Add lan...,TC-E03
65,Total landings. Use only when there is more th...,TL-1-E03MT
66,Total landings in subarea,TL-A3-MT


In [38]:
fish = (time_series
    .rename(tsunique = "tsid")
    .join(metrics, "tsunique")
    .join(stock, ["stockid", "stocklong"])
    .join(assessment, "assessid")
)

cod_catch = (fish
.filter(_.tscategory == 'CATCH or LANDINGS')
.filter(_.commonname == "Atlantic cod")
)

#cod_catch.head().execute()

fish.group_by(_.tslong, _.tsunique).agg(n = _.count()).order_by(_.n.desc()).execute()

#fish.columns # to view the columns


,tslong,tsunique,n
0,Catch divided by mean catch,CdivMEANC-ratio,138234
1,"General total Catch (TC then TL, MT units only)",TCbest-MT,127713
2,Total catch (i.e. landings + discards. Add lan...,TC-MT,86609
3,Total landings,TL-MT,72643
4,General biomass time series preferentially rel...,BdivBmgtpref-dimensionless,60802
...,...,...,...
68,Total catch (i.e. landings + discards. Add lan...,TC-2-MT,20
69,Total catch (i.e. landings + discards. Add lan...,TC-3-MT,20
70,Total allowable catch,TAC-2-MT,16
71,Total allowable catch,TAC-1-MT,15


# Excersise 2: Global Fisheries 

## Stock Collapses

We seek to replicate the temporal trend in stock declines shown in [Worm et al 2006](http://doi.org/10.1126/science.1132294):

![](https://espm-157.github.io/website-r/img/worm2006.jpg)